# **ANALISIS OUTLIERS DATAFRAMES**

In [9]:
# Importamos posibles librerias que utilizaremos
import numpy as np
import pandas as pd
import feather

In [10]:
# Cargamos los dataframes
df_listings = pd.read_feather (r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/2. data_sin_nulos/df_listings.feather')
df_reviews = pd.read_feather (r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/2. data_sin_nulos/df_reviews.feather')
df_calendar = pd.read_feather(r'/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/2. data_sin_nulos/df_calendar.feather')

In [11]:
# Mostrar todas las columnas y filas de nuestro dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Configuración del formato de numeros
pd.options.display.float_format = '{:.2f}'.format

In [12]:
# Hacemos una copia de los dataframe antes de eliminar o modificar los datos
df_listings_copy = df_listings.copy()
df_reviews_copy = df_reviews.copy()
df_calendar_copy = df_calendar.copy()

In [13]:
# Eliminamos valores nulos si aún quedan
df_calendar = df_calendar.dropna()
df_listings = df_listings.dropna()
df_reviews = df_reviews.dropna()

# <span style="color:red"><b>**ANALISIS OUTLIERS**</b></span>
Disponemos de los siguientes dataframe:
- df_calendar
- df_reviews
- df_listings

### **DF_CALENDAR**

In [14]:
df_calendar.head()

,listing_id,date,available,price,minimum_nights,maximum_nights
0,97529,2024-03-20,f,47.00,120,1125
1,360863,2024-03-20,f,50.00,31,150
2,360863,2024-03-21,f,50.00,31,150
3,360863,2024-03-22,f,50.00,31,150
4,360863,2024-03-23,f,50.00,31,150


In [15]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6759363 entries, 0 to 6759362
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           float64
 4   minimum_nights  int64  
 5   maximum_nights  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 309.4+ MB


In [16]:
# Vemos los datos estadísticas de las variables numéricas de calendar
df_calendar.describe()


,listing_id,price,minimum_nights,maximum_nights
count,6759363.00,6759363.00,6759363.00,6759363.00
mean,381118365036741824.00,237.18,17.11,62264.46
std,444672572982365248.00,1931.88,39.95,11504599.86
min,17475.00,9.00,1.00,1.00
25%,23317712.00,50.00,2.00,330.00
50%,50219794.00,99.00,4.00,365.00
75%,859905687612812544.00,185.00,31.00,1125.00
max,1115690065090507776.00,231858.00,2705.00,2147483647.00


Viendo los datos podemos ver como

In [17]:
# Arreglar outliers
def arreglar_outliers_simple(df):
    # Creamos un nuevo DataFrame para almacenar los valores tratados
    df_cleaned = df.copy()

    # Iteramos sobre cada columna del DataFrame
    for col in df_cleaned.columns:
        # Calculamos los percentiles 25 y 75 (Q1 y Q3) -- ¿recordáis lo que son en un boxplot?
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)

        # Calculamos el rango intercuartílico (IQR)
        IQR = Q3 - Q1

        # Calculamos los límites del intervalo
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        # Reemplazamos los valores atípicos por los límites del  (el upper si está por encima, el lower si está por debajo)
        df_cleaned[col] = df_cleaned[col].apply(lambda x: lower_limit if x < lower_limit else (upper_limit if x > upper_limit else x))

    return df_cleaned

In [18]:
df_calendar['price'] = arreglar_outliers_simple(df_calendar[['price']])['price']
df_calendar['minimum_nights'] = arreglar_outliers_simple(df_calendar[['minimum_nights']])['minimum_nights']
df_calendar['maximum_nights'] = arreglar_outliers_simple(df_calendar[['maximum_nights']])['maximum_nights']

In [19]:
df_calendar.describe().T

,count,mean,std,min,25%,50%,75%,max
listing_id,6759363.00,381118365036741824.00,444672572982365248.00,17475.00,23317712.00,50219794.00,859905687612812544.00,1115690065090507776.00
price,6759363.00,136.98,112.88,9.00,50.00,99.00,185.00,387.50
minimum_nights,6759363.00,15.09,16.60,1.00,2.00,4.00,31.00,74.50
maximum_nights,6759363.00,629.26,450.68,1.00,330.00,365.00,1125.00,2317.50


### **DF_REVIEWS**

In [20]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,360863,1124282,2012-04-13,1853183,Anna,We stayed at Humberto's apartment for 9 days ...
1,17475,63134,2010-07-10,141807,Zoltan,"Excellent host, beautiful apartment, great loc..."
2,17475,83802,2010-08-24,68687,John,"Luca is very personable, and the views are exa..."
3,17475,98998,2010-09-17,96055,Xiaoyang,I am glad we found Luca's apt for our stay in ...
4,17475,105280,2010-09-26,171084,Amy,I should start by saying we really did not wan...


In [21]:
df_reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [22]:
df_reviews.describe()

,listing_id,id,reviewer_id
count,833583.00,833583.00,833583.00
mean,76604514525124592.00,446930567729488704.00,149619424.13
std,233454018874632000.00,426182947961827392.00,149045096.71
min,17475.00,63134.00,3.00
25%,3144104.00,334872141.50,30953570.50
50%,16425528.00,512072265275724224.00,94212857.00
75%,36996478.00,865020248351326464.00,225835139.00
max,1110775765657684096.00,1116547371400319104.00,567689335.00


No es necesario arreglar ningun outliers

## **DF_LISTINGS**

In [23]:
df_listings.head()

,id,listing_url,name,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,bathrooms,amenities,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,calculated_host_listings_count
0,360863.00,https://www.airbnb.com/rooms/360863,SPECIAL OFFER @ COSY AND CHARMING,la Barceloneta,Ciutat Vella,Private room in rental unit,Private room,41.38,2.20,173.00,31.00,150.00,2.00,2.18,2.73,1.14,"[""Cleaning available during stay"", ""Baking she...",32.00,0.00,0.22,4.73,4.84,4.94,4.71,4.74,t,6%,2.00,0.00,2.00
1,17475.00,https://www.airbnb.com/rooms/17475,Attic Sagrada Familia,la Dreta de l'Eixample,Eixample,Entire rental unit,Entire home/apt,41.40,2.17,165.00,3.00,90.00,3.00,1.00,1.00,1.00,"[""Microwave"", ""Outdoor dining area"", ""Private ...",29.00,12.00,0.17,4.48,4.23,4.73,4.82,4.50,f,100%,2.00,7.00,1.00
2,18674.00,https://www.airbnb.com/rooms/18674,Huge flat for 8 people close to Sagrada Familia,la Sagrada Família,Eixample,Entire rental unit,Entire home/apt,41.41,2.17,210.00,1.00,1125.00,8.00,3.00,6.00,2.00,"[""Paid parking on premises"", ""Private patio or...",40.00,6.00,0.30,4.33,4.62,4.67,4.77,4.28,f,97%,44.00,252.00,28.00
3,97529.00,https://www.airbnb.com/rooms/97529,Furnished one bedroom apartment with bathroom,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.41,2.17,170.00,120.00,1125.00,2.00,1.00,4.27,1.27,"[""Microwave"", ""Paid parking off premises"", ""Pr...",67.00,0.00,0.45,4.74,4.86,4.91,4.45,4.71,f,20%,1.00,0.00,1.00
4,110041.00,https://www.airbnb.com/rooms/110041,002. Alió 2 Apartment,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.40,2.17,94.00,1.00,1125.00,4.00,1.00,2.00,1.00,"[""Microwave"", ""Iron"", ""Dedicated workspace"", ""...",54.00,18.00,0.38,4.17,4.24,4.41,4.65,4.11,f,100%,114.00,226.00,95.00


In [24]:
df_listings.describe().T

,count,mean,std,min,25%,50%,75%,max
id,18519.00,381114305448584192.00,444683917798170368.00,17475.00,23318863.50,50219794.00,859860715961919232.00,1115690065090507776.00
latitude,18519.00,41.39,0.01,41.35,41.38,41.39,41.40,41.46
longitude,18519.00,2.17,0.02,2.09,2.16,2.17,2.18,2.23
price,18519.00,147.29,213.46,10.00,69.00,124.00,178.00,11999.00
minimum_nights,18519.00,15.50,32.85,1.00,1.00,3.00,31.00,1124.00
maximum_nights,18519.00,554.09,433.13,1.00,300.00,365.00,1125.00,3000.00
accommodates,18519.00,3.36,2.15,1.00,2.00,3.00,4.00,16.00
bedrooms,18519.00,1.82,1.13,0.00,1.00,1.82,2.00,18.00
beds,18519.00,2.47,1.72,0.00,1.00,2.00,3.00,30.00
bathrooms,18519.00,1.40,0.65,0.00,1.00,1.00,1.86,10.00


In [25]:
# Arreglar outliers
def arreglar_outliers_simple(df):
    # Creamos un nuevo DataFrame para almacenar los valores tratados
    df_cleaned = df.copy()

    # Iteramos sobre cada columna del DataFrame
    for col in df_cleaned.columns:
        # Calculamos los percentiles 25 y 75 (Q1 y Q3) -- ¿recordáis lo que son en un boxplot?
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)

        # Calculamos el rango intercuartílico (IQR)
        IQR = Q3 - Q1

        # Calculamos los límites del intervalo
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR

        # Reemplazamos los valores atípicos por los límites del  (el upper si está por encima, el lower si está por debajo)
        df_cleaned[col] = df_cleaned[col].apply(lambda x: lower_limit if x < lower_limit else (upper_limit if x > upper_limit else x))

    return df_cleaned

In [26]:
columns_to_fix = ['price', 'minimum_nights', 'maximum_nights', 'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month', 'review_scores_rating', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'host_listings_count','availability_365', 'calculated_host_listings_count','bathrooms']

for column in columns_to_fix:
    df_listings[column] = arreglar_outliers_simple(df_listings[[column]])[column]


In [27]:
df_listings.describe().T

,count,mean,std,min,25%,50%,75%,max
id,18519.00,381114305448584192.00,444683917798170368.00,17475.00,23318863.50,50219794.00,859860715961919232.00,1115690065090507776.00
latitude,18519.00,41.39,0.01,41.35,41.38,41.39,41.40,41.46
longitude,18519.00,2.17,0.02,2.09,2.16,2.17,2.18,2.23
price,18519.00,134.23,81.36,10.00,69.00,124.00,178.00,341.50
minimum_nights,18519.00,14.10,16.06,1.00,1.00,3.00,31.00,76.00
maximum_nights,18519.00,554.03,432.79,1.00,300.00,365.00,1125.00,2362.50
accommodates,18519.00,3.24,1.81,1.00,2.00,3.00,4.00,7.00
bedrooms,18519.00,1.74,0.88,0.00,1.00,1.82,2.00,3.50
beds,18519.00,2.40,1.43,0.00,1.00,2.00,3.00,6.00
bathrooms,18519.00,1.38,0.54,0.00,1.00,1.00,1.86,3.16


Ya tenemos todos los dataframes arreglados tanto de valores nulos como de valores atipicos o outliers. Ahora vamos a valorar si juntarlos o no.

# **VALORAR SI JUNTAR DATAFRAME**
En este apartamos vamos a ver que columnas vamos a tener en cuenta para poder unir los cuatro dataframes en una sola hoja.

#### DF_CALENDAR

In [28]:
df_calendar.head()

,listing_id,date,available,price,minimum_nights,maximum_nights
0,97529,2024-03-20,f,47.00,74.50,1125.00
1,360863,2024-03-20,f,50.00,31.00,150.00
2,360863,2024-03-21,f,50.00,31.00,150.00
3,360863,2024-03-22,f,50.00,31.00,150.00
4,360863,2024-03-23,f,50.00,31.00,150.00


In [29]:
df_calendar.columns

Index(['listing_id', 'date', 'available', 'price', 'minimum_nights',
       'maximum_nights'],
      dtype='object')

In [30]:
# Comprobamos que este todo bien
df_calendar.head()

,listing_id,date,available,price,minimum_nights,maximum_nights
0,97529,2024-03-20,f,47.00,74.50,1125.00
1,360863,2024-03-20,f,50.00,31.00,150.00
2,360863,2024-03-21,f,50.00,31.00,150.00
3,360863,2024-03-22,f,50.00,31.00,150.00
4,360863,2024-03-23,f,50.00,31.00,150.00


#### DF_REVIEWS

In [31]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,360863,1124282,2012-04-13,1853183,Anna,We stayed at Humberto's apartment for 9 days ...
1,17475,63134,2010-07-10,141807,Zoltan,"Excellent host, beautiful apartment, great loc..."
2,17475,83802,2010-08-24,68687,John,"Luca is very personable, and the views are exa..."
3,17475,98998,2010-09-17,96055,Xiaoyang,I am glad we found Luca's apt for our stay in ...
4,17475,105280,2010-09-26,171084,Amy,I should start by saying we really did not wan...


In [32]:
df_reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [33]:
# Vamos a eliminar las columnas de id. y reviewer_id
df_reviews = df_reviews.drop(columns=['id', 'date','reviewer_id'])

In [34]:
# Comprobamos que se ha llevado a cabo correctamente
df_reviews.head()

,listing_id,reviewer_name,comments
0,360863,Anna,We stayed at Humberto's apartment for 9 days ...
1,17475,Zoltan,"Excellent host, beautiful apartment, great loc..."
2,17475,John,"Luca is very personable, and the views are exa..."
3,17475,Xiaoyang,I am glad we found Luca's apt for our stay in ...
4,17475,Amy,I should start by saying we really did not wan...


Ya tenemos el dataframe sin las columnas que no vamos a necesitar para nuestro analisis.

#### DF_LISTINGS

In [35]:
df_listings.head()

,id,listing_url,name,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,latitude,longitude,price,minimum_nights,maximum_nights,accommodates,bedrooms,beds,bathrooms,amenities,number_of_reviews,number_of_reviews_ltm,reviews_per_month,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value,host_is_superhost,host_response_rate,host_listings_count,availability_365,calculated_host_listings_count
0,360863.00,https://www.airbnb.com/rooms/360863,SPECIAL OFFER @ COSY AND CHARMING,la Barceloneta,Ciutat Vella,Private room in rental unit,Private room,41.38,2.20,173.00,31.00,150.00,2.00,2.18,2.73,1.14,"[""Cleaning available during stay"", ""Baking she...",32.00,0.00,0.22,4.73,4.84,4.94,4.71,4.74,t,6%,2.00,0.00,2.00
1,17475.00,https://www.airbnb.com/rooms/17475,Attic Sagrada Familia,la Dreta de l'Eixample,Eixample,Entire rental unit,Entire home/apt,41.40,2.17,165.00,3.00,90.00,3.00,1.00,1.00,1.00,"[""Microwave"", ""Outdoor dining area"", ""Private ...",29.00,12.00,0.17,4.48,4.23,4.73,4.82,4.50,f,100%,2.00,7.00,1.00
2,18674.00,https://www.airbnb.com/rooms/18674,Huge flat for 8 people close to Sagrada Familia,la Sagrada Família,Eixample,Entire rental unit,Entire home/apt,41.41,2.17,210.00,1.00,1125.00,7.00,3.00,6.00,2.00,"[""Paid parking on premises"", ""Private patio or...",40.00,6.00,0.30,4.33,4.62,4.67,4.77,4.28,f,97%,44.00,252.00,28.00
3,97529.00,https://www.airbnb.com/rooms/97529,Furnished one bedroom apartment with bathroom,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.41,2.17,170.00,76.00,1125.00,2.00,1.00,4.27,1.27,"[""Microwave"", ""Paid parking off premises"", ""Pr...",67.00,0.00,0.45,4.74,4.86,4.91,4.45,4.71,f,20%,1.00,0.00,1.00
4,110041.00,https://www.airbnb.com/rooms/110041,002. Alió 2 Apartment,el Camp d'en Grassot i Gràcia Nova,Gràcia,Entire rental unit,Entire home/apt,41.40,2.17,94.00,1.00,1125.00,4.00,1.00,2.00,1.00,"[""Microwave"", ""Iron"", ""Dedicated workspace"", ""...",54.00,18.00,0.38,4.17,4.24,4.41,4.65,4.11,f,100%,97.00,226.00,78.50


In [36]:
df_listings.columns

Index(['id', 'listing_url', 'name', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'property_type', 'room_type',
       'latitude', 'longitude', 'price', 'minimum_nights', 'maximum_nights',
       'accommodates', 'bedrooms', 'beds', 'bathrooms', 'amenities',
       'number_of_reviews', 'number_of_reviews_ltm', 'reviews_per_month',
       'review_scores_rating', 'review_scores_cleanliness',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'host_is_superhost', 'host_response_rate',
       'host_listings_count', 'availability_365',
       'calculated_host_listings_count'],
      dtype='object')

In [46]:
# Guardamos los dataframes en cada una de sus columnas
#import os

# Definiimos el lugar donde vamos a guardar cada uno de los dataframes
#base_path = '/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers'
#folder_path = os.path.join(base_path)

# Guarda cada DataFrame como un archivo CSV en la carpeta específica
#df_listings.to_csv(os.path.join(folder_path, 'df_listings.csv'), index=False)
#df_reviews.to_csv(os.path.join(folder_path, 'df_reviews'), index=False)
#df_calendar.to_csv(os.path.join(folder_path, 'df_calendar'), index=False)


In [37]:
# codigo para guardar los archivos en feather que ocupan menos espacio
"""
import os
import feather

# Definimos la carpeta específica donde vamos a guardar los DataFrames sin valores nulos
base_path = '/Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers'
folder_path = os.path.join(base_path)

# Crea la carpeta si no existe
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Guarda cada DataFrame como un archivo Feather en la carpeta específica
feather.write_dataframe(df_calendar, os.path.join(folder_path, 'df_calendar.feather'))
feather.write_dataframe(df_listings, os.path.join(folder_path, 'df_listings.feather'))
feather.write_dataframe(df_reviews, os.path.join(folder_path, 'df_reviews.feather'))

print(f"Archivos Feather guardados en la carpeta: {folder_path}")
""" 

Archivos Feather guardados en la carpeta: /Users/juliobrionesmorales/Documents/GitHub/Airbnb_Barcelona/data/3. data_sin_outliers
